# Análisis de Transacciones - Datathon
## Notebook de Limpieza y Visualización de Datos

In [14]:
import polars as pl

## 1. CARGA INICIAL DE DATOS

In [ ]:
import polars as pl

# Modificación en la sección 1. CARGA INICIAL DE DATOS
df = pl.read_csv(
    "../backend/data/base_transacciones_final.csv",
    dtypes={
        "id": pl.Utf8,
        "comercio": pl.Utf8,
        "giro_comercio": pl.Utf8,
        "tipo_venta": pl.Utf8,
        "monto": pl.Float64,
        # Convierte 'fecha' a tipo fecha-hora directamente al cargar
        "fecha": pl.Datetime
    },
    # Agrega el argumento parse_dates=True si tu archivo tiene fechas en un formato estándar
    # o especifica el formato si es necesario con try_parse_dates=True
    try_parse_dates=True
)


/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_51463/4036598233.py:4: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df = pl.read_csv(


## 2. EXPLORACIÓN INICIAL DEL DATASET

In [16]:
# 2.1 Información General
# Forma y tipos
print("Shape:", df.shape)
print("Dtypes:", df.dtypes)

# Ver primeras filas
print("\nPrimeras 5 filas:")
print(df.head())

# 2.2 Estadísticas Descriptivas
# Estadísticas de monto y conteos únicos
stats = df.select([
    pl.col("monto").mean().alias("monto_mean"),
    pl.col("monto").median().alias("monto_median"),
    pl.col("monto").std().alias("monto_std"),
    pl.col("monto").min().alias("min"),
    pl.col("monto").max().alias("max"),
    pl.col("id").n_unique().alias("unique_ids"),
    pl.col("comercio").n_unique().alias("unique_merchants")
    
])
print("Estadísticas:")
print(stats)

# 2.3 Análisis de Datos Faltantes
# Datos faltantes por columna
print("Datos faltantes por columna:")
print(df.null_count())

Shape: (346011, 6)
Dtypes: [String, String, String, String, String, Float64]

Primeras 5 filas:
shape: (5, 6)
┌──────────────────────────┬────────────┬──────────┬──────────────────────────┬────────────┬───────┐
│ id                       ┆ fecha      ┆ comercio ┆ giro_comercio            ┆ tipo_venta ┆ monto │
│ ---                      ┆ ---        ┆ ---      ┆ ---                      ┆ ---        ┆ ---   │
│ str                      ┆ str        ┆ str      ┆ str                      ┆ str        ┆ f64   │
╞══════════════════════════╪════════════╪══════════╪══════════════════════════╪════════════╪═══════╡
│ 91477f382c3cf63ab5cd9263 ┆ 2022-01-02 ┆ AMAZON   ┆ COMERCIOS ELECTRONICOS   ┆ digital    ┆ 5.99  │
│ b50210…                  ┆            ┆          ┆ (VTAS P…                 ┆            ┆       │
│ 91477f382c3cf63ab5cd9263 ┆ 2022-01-05 ┆ RAPPI    ┆ SERVICIOS EMPRESARIALES  ┆ digital    ┆ 13.01 │
│ b50210…                  ┆            ┆          ┆ - NO C…                  ┆   

## 3. ANÁLISIS DE DUPLICADOS

In [17]:
# 3.1 Detección de Duplicados Exactos
print("=== ANÁLISIS DE DUPLICADOS EXACTOS ===")
total_original = df.height

# Verificar duplicados exactos: TODAS las columnas idénticas
print("Verificando duplicados donde TODAS las columnas son idénticas...")

# Método robusto: group by todas las columnas
conteo_exacto = df.group_by(df.columns).agg(pl.len().alias("repeticiones"))
duplicados_exactos = conteo_exacto.filter(pl.col("repeticiones") > 1)

# Calcular estadísticas
grupos_duplicados = duplicados_exactos.height
total_filas_duplicadas = duplicados_exactos.select((pl.col("repeticiones") - 1).sum()).item() if grupos_duplicados > 0 else 0

print(f"Filas originales: {total_original:,}")
print(f"Grupos de filas EXACTAMENTE idénticas: {grupos_duplicados:,}")
print(f"Total de filas duplicadas exactas: {total_filas_duplicadas:,}")

# 3.2 Análisis de Duplicados por Comercio
if grupos_duplicados > 0:
    print("\n=== ANÁLISIS DE DUPLICADOS POR COMERCIO ===")
    
    # Agregar columna de comercio Y giro_comercio a los duplicados exactos y calcular estadísticas
    duplicados_por_comercio = (
        duplicados_exactos
        .select([
            pl.col("comercio"),
            pl.col("giro_comercio"),
            (pl.col("repeticiones") - 1).alias("num_duplicados"),  # Solo contar las copias extra
            pl.col("id").alias("cliente_id")
        ])
        .group_by(["comercio", "giro_comercio"])
        .agg([
            pl.col("num_duplicados").sum().alias("total_duplicados"),
            pl.col("cliente_id").n_unique().alias("clientes_afectados")
        ])
        .with_columns([
            (pl.col("total_duplicados") / total_filas_duplicadas * 100).alias("porcentaje_de_duplicados")
        ])
        .sort("total_duplicados", descending=True)
    )
    
    # EXPORTAR A CSV
    duplicados_por_comercio.write_csv("../data/duplicados_por_comercio.csv")
    print(f"✅ Exportado análisis completo a: ../data/duplicados_por_comercio.csv")

    # 3.3 Top Comercios con Duplicados
    print("COMERCIOS CON MÁS DUPLICADOS:")
    print("(comercio | giro_comercio | duplicados | % del total | clientes afectados)")
    print(f"Total de comercios con duplicados: {duplicados_por_comercio.height}")
    print("\nPrimeros 20:")
    print(duplicados_por_comercio.head(20))
    
    # Top 10 comercios problemáticos
    print(f"\n=== TOP 10 COMERCIOS MÁS PROBLEMÁTICOS ===")
    top_comercios_problematicos = duplicados_por_comercio.head(10)
    print(top_comercios_problematicos)

    # 3.4 Estadísticas Resumidas de Comercios Problemáticos
    print(f"\n=== ESTADÍSTICAS DE COMERCIOS PROBLEMÁTICOS ===")
    total_comercios_con_duplicados = duplicados_por_comercio.height
    comercios_top_5 = duplicados_por_comercio.head(5)
    duplicados_top_5 = comercios_top_5.select("total_duplicados").sum().item()
    porcentaje_top_5 = (duplicados_top_5 / total_filas_duplicadas) * 100
    
    print(f"Comercios con duplicados: {total_comercios_con_duplicados}")
    print(f"Top 5 comercios representan: {duplicados_top_5:,} duplicados ({porcentaje_top_5:.1f}% del total)")

    # 3.5 Ejemplo de Transacción Duplicada
    # Mostrar UN ejemplo de transacción duplicada (cualquiera)
    print("\n=== EJEMPLO DE TRANSACCIÓN DUPLICADA ===")
    primer_grupo = duplicados_exactos.head(1)
    repeticiones_ejemplo = primer_grupo.select("repeticiones").item()
    
    # Obtener los valores del primer grupo duplicado
    valores_ejemplo = primer_grupo.drop("repeticiones").row(0)
    
    # Filtrar para mostrar todas las repeticiones de este ejemplo
    condiciones = []
    for i, col in enumerate(df.columns):
        condiciones.append(pl.col(col) == valores_ejemplo[i])
    
    filas_ejemplo = df.filter(pl.all_horizontal(condiciones))
    print(f"Transacción que aparece {repeticiones_ejemplo} veces:")
    print(filas_ejemplo)
    
    print("\n=== TOP 10 GRUPOS CON MÁS REPETICIONES EXACTAS ===")
    top_duplicados = duplicados_exactos.sort("repeticiones", descending=True).head(10)
    print(top_duplicados)

else:
    print("✅ No hay duplicados exactos (todas las columnas idénticas)")

# 3.6 Resumen de Duplicados
print(f"\n=== RESUMEN ===")
if total_filas_duplicadas > 0:
    print(f"Se encontraron {total_filas_duplicadas:,} filas que son duplicados exactos")
    print("✅ Archivo 'duplicados_por_comercio.csv' creado con análisis completo")
    print("(No se eliminaron - solo análisis)")
else:
    print("No hay duplicados exactos para eliminar")

=== ANÁLISIS DE DUPLICADOS EXACTOS ===
Verificando duplicados donde TODAS las columnas son idénticas...
Filas originales: 346,011
Grupos de filas EXACTAMENTE idénticas: 12,822
Total de filas duplicadas exactas: 18,250

=== ANÁLISIS DE DUPLICADOS POR COMERCIO ===
✅ Exportado análisis completo a: ../data/duplicados_por_comercio.csv
COMERCIOS CON MÁS DUPLICADOS:
(comercio | giro_comercio | duplicados | % del total | clientes afectados)
Total de comercios con duplicados: 226

Primeros 20:
shape: (20, 5)
┌──────────────┬─────────────────────┬──────────────────┬────────────────────┬─────────────────────┐
│ comercio     ┆ giro_comercio       ┆ total_duplicados ┆ clientes_afectados ┆ porcentaje_de_dupli │
│ ---          ┆ ---                 ┆ ---              ┆ ---                ┆ cados               │
│ str          ┆ str                 ┆ u32              ┆ u32                ┆ ---                 │
│              ┆                     ┆                  ┆                    ┆ f64         

## 4. NORMALIZACIÓN DE NOMBRES DE COMERCIOS

In [18]:
# 4.1 Definición de Función de Normalización
print("=== NORMALIZACIÓN DE CASOS ESPECÍFICOS IDENTIFICADOS ===")

def normalizar_casos_especificos(comercio_raw, giro_comercio):
    """
    Solo normaliza los 3 casos específicos identificados donde hay 
    variaciones del mismo merchant en el mismo giro_comercio
    """
    if comercio_raw is None:
        return "DESCONOCIDO"
    
    comercio = str(comercio_raw).upper().strip()
    
    # Caso 1: 7-Eleven en TIENDAS DE CONVENIENCIA
    if (comercio in ['7 ELEVEN', '7ELEVEN'] and 
        giro_comercio == 'TIENDAS DE CONVENIENCIA, MINISUPER'):
        return "7ELEVEN"
    
    # Caso 2: DidiFood en LIMOSINAS (TAXIS)  
    if (comercio in ['DIDI FOOD', 'DIDIFOOD'] and 
        giro_comercio == 'LIMOSINAS, (TAXIS)'):
        return "DIDIFOOD"
    
    # Caso 3: DidiFood en COMIDA RAPIDA
    if (comercio in ['DIDI FOOD', 'DIDIFOOD'] and 
        giro_comercio == 'COMIDA RAPIDA'):
        return "DIDIFOOD"
    
    # Todo lo demás queda igual
    return comercio

# 4.2 Aplicación de Normalización
# Aplicar normalización específica
df = df.with_columns([
    pl.struct(["comercio", "giro_comercio"])
    .map_elements(lambda x: normalizar_casos_especificos(x["comercio"], x["giro_comercio"]), return_dtype=pl.Utf8)
    .alias("merchant_std")
])

# 4.3 Verificación de Normalización
# Verificar exactamente qué se normalizó
print("=== VERIFICACIÓN DE CASOS NORMALIZADOS ===")
casos_normalizados = df.filter(
    pl.col("comercio") != pl.col("merchant_std")
).group_by(["comercio", "merchant_std", "giro_comercio"]).agg(pl.len().alias("transacciones"))

print("Casos que fueron normalizados:")
print(casos_normalizados)

# Estadísticas de impacto
merchants_antes = df.select(pl.col("comercio").n_unique()).item()
merchants_despues = df.select(pl.col("merchant_std").n_unique()).item()
reduccion = merchants_antes - merchants_despues

print(f"\n=== IMPACTO FINAL ===")
print(f"Merchants únicos antes: {merchants_antes:,}")
print(f"Merchants únicos después: {merchants_despues:,}")
print(f"Reducción: {reduccion:,} merchants (exactamente 3 casos normalizados)")

print("\n✅ Normalización completada para los 3 casos específicos identificados")

# OPCIONAL: Exportar el DataFrame normalizado a parquet para uso futuro
df.write_parquet("../data/transactions_clean.parquet")
print("✅ DataFrame normalizado exportado a: ../data/transactions_clean.parquet")



=== NORMALIZACIÓN DE CASOS ESPECÍFICOS IDENTIFICADOS ===
=== VERIFICACIÓN DE CASOS NORMALIZADOS ===
Casos que fueron normalizados:
shape: (3, 4)
┌───────────┬──────────────┬─────────────────────────────────┬───────────────┐
│ comercio  ┆ merchant_std ┆ giro_comercio                   ┆ transacciones │
│ ---       ┆ ---          ┆ ---                             ┆ ---           │
│ str       ┆ str          ┆ str                             ┆ u32           │
╞═══════════╪══════════════╪═════════════════════════════════╪═══════════════╡
│ 7 ELEVEN  ┆ 7ELEVEN      ┆ TIENDAS DE CONVENIENCIA, MINIS… ┆ 10425         │
│ DIDI FOOD ┆ DIDIFOOD     ┆ COMIDA RAPIDA                   ┆ 4583          │
│ DIDI FOOD ┆ DIDIFOOD     ┆ LIMOSINAS, (TAXIS)              ┆ 42            │
└───────────┴──────────────┴─────────────────────────────────┴───────────────┘

=== IMPACTO FINAL ===
Merchants únicos antes: 97
Merchants únicos después: 97
Reducción: 0 merchants (exactamente 3 casos normalizados)

✅ Norm

## 5. ANÁLISIS DEL DATASET LIMPIO

In [25]:
# 5.1 Usar el DataFrame ya procesado
print("=== PREPARANDO DATASET LIMPIO ===")

# Usar el DataFrame que ya procesamos (df tiene la normalización aplicada)
df_clean = df

print(f"Dataset cargado exitosamente")
print(f"Shape: {df_clean.shape}")
print(f"Columnas: {df_clean.columns}")

# 5.2 Overview General
print(f"\n=== OVERVIEW GENERAL ===")
print(f"Total transacciones: {df_clean.height:,}")
print(f"Total clientes únicos: {df_clean.select(pl.col('id').n_unique()).item():,}")
print(f"Total merchants únicos (original): {df_clean.select(pl.col('comercio').n_unique()).item():,}")

# Verificar si tiene merchant_std
if "merchant_std" in df_clean.columns:
    print(f"Total merchants únicos (normalizado): {df_clean.select(pl.col('merchant_std').n_unique()).item():,}")

# Rango de fechas
print(f"Fecha mínima: {df_clean.select(pl.col('fecha').min()).item()}")
print(f"Fecha máxima: {df_clean.select(pl.col('fecha').max()).item()}")

# 5.3 Información de Tipos de Datos
print(f"\n=== TIPOS DE DATOS ===")
for col, dtype in zip(df_clean.columns, df_clean.dtypes):
    print(f"{col}: {dtype}")

# Verificar datos faltantes
print(f"\n=== DATOS FALTANTES ===")
print(df_clean.null_count())

# Primeras 10 filas
print(f"\n=== PRIMERAS 10 FILAS ===")
print(df_clean.head(10))

# 5.4 Estadísticas de Montos
print(f"\n=== ESTADÍSTICAS DE MONTOS ===")
stats_monto = df_clean.select([
    pl.col("monto").min().alias("min"),
    pl.col("monto").quantile(0.25).alias("q25"),
    pl.col("monto").median().alias("mediana"),
    pl.col("monto").mean().alias("promedio"),
    pl.col("monto").quantile(0.75).alias("q75"),
    pl.col("monto").quantile(0.95).alias("q95"),
    pl.col("monto").max().alias("max"),
    pl.col("monto").std().alias("std")
])
print(stats_monto)

# 5.5 Distribución por Tipo de Venta
print(f"\n=== DISTRIBUCIÓN POR TIPO DE VENTA ===")
dist_tipo_venta = df_clean.group_by("tipo_venta").agg([
    pl.len().alias("transacciones"),
    (pl.len() / df_clean.height * 100).alias("porcentaje")
]).sort("transacciones", descending=True)
print(dist_tipo_venta)

# 5.6 Top 15 Giros Comerciales
print(f"\n=== TOP 15 GIROS COMERCIALES ===")
top_giros = df_clean.group_by("giro_comercio").agg([
    pl.len().alias("transacciones"),
    (pl.len() / df_clean.height * 100).alias("porcentaje")
]).sort("transacciones", descending=True).head(15)
print(top_giros)

# 5.7 Top 15 Merchants
print(f"\n=== TOP 15 MERCHANTS ===")
merchant_col = "merchant_std" if "merchant_std" in df_clean.columns else "comercio"
top_merchants = df_clean.group_by(merchant_col).agg([
    pl.len().alias("transacciones"),
    (pl.len() / df_clean.height * 100).alias("porcentaje"),
    pl.col("monto").mean().alias("monto_promedio")
]).sort("transacciones", descending=True).head(15)
print(top_merchants)

df.write_parquet("../data/transactions_clean.parquet")
print("✅ DataFrame normalizado exportado a: ../data/transactions_clean.parquet")

# 5.8 Distribución Temporal
print(f"\n=== DISTRIBUCIÓN TEMPORAL (POR MES) ===")
dist_mensual = df_clean.with_columns([
    pl.col("fecha").dt.strftime("%Y-%m").alias("año_mes")
]).group_by("año_mes").agg([
    pl.len().alias("transacciones"),
    pl.col("monto").sum().alias("monto_total")
]).sort("año_mes")
print(dist_mensual)

print(f"\n✅ Dataset transactions_clean.parquet visualizado completamente")

=== PREPARANDO DATASET LIMPIO ===
Dataset cargado exitosamente
Shape: (346011, 6)
Columnas: ['id', 'fecha', 'comercio', 'giro_comercio', 'tipo_venta', 'monto']

=== OVERVIEW GENERAL ===
Total transacciones: 346,011
Total clientes únicos: 1,000
Total merchants únicos (original): 97
Fecha mínima: 2022-01-01 00:00:00
Fecha máxima: 2023-01-30 00:00:00

=== TIPOS DE DATOS ===
id: String
fecha: Datetime(time_unit='us', time_zone=None)
comercio: String
giro_comercio: String
tipo_venta: String
monto: Float64

=== DATOS FALTANTES ===
shape: (1, 6)
┌─────┬───────┬──────────┬───────────────┬────────────┬───────┐
│ id  ┆ fecha ┆ comercio ┆ giro_comercio ┆ tipo_venta ┆ monto │
│ --- ┆ ---   ┆ ---      ┆ ---           ┆ ---        ┆ ---   │
│ u32 ┆ u32   ┆ u32      ┆ u32           ┆ u32        ┆ u32   │
╞═════╪═══════╪══════════╪═══════════════╪════════════╪═══════╡
│ 0   ┆ 0     ┆ 0        ┆ 5588          ┆ 0          ┆ 0     │
└─────┴───────┴──────────┴───────────────┴────────────┴───────┘

=== PR

## 6. EXPORTACIÓN DEL DATASET LIMPIO

In [26]:
# 6.1 Exportar a CSV
print("=== EXPORTANDO DATASET LIMPIO A CSV ===")

# Exportar el dataset limpio a CSV
df_clean.write_csv("../data/transactions_clean.csv")

print(f"✅ Dataset exportado exitosamente a: ../data/transactions_clean.csv")
print(f"   - Total de filas exportadas: {df_clean.height:,}")
print(f"   - Total de columnas: {len(df_clean.columns)}")
print(f"   - Tamaño aproximado: {df_clean.height * len(df_clean.columns) * 50 / 1_000_000:.1f} MB (estimado)")




=== EXPORTANDO DATASET LIMPIO A CSV ===
✅ Dataset exportado exitosamente a: ../data/transactions_clean.csv
   - Total de filas exportadas: 346,011
   - Total de columnas: 6
   - Tamaño aproximado: 103.8 MB (estimado)


In [17]:
import polars as pl
from IPython.display import display

# 1. Leer el CSV de transacciones limpias
df = pl.read_csv(
    "../data/transactions_clean.csv",
    dtypes={
        "id":           pl.Utf8,
        "fecha":        pl.Utf8,    # ya lo convertirás a Date si lo necesitas luego
        "comercio":     pl.Utf8,
        "giro_comercio":pl.Utf8,
        "tipo_venta":   pl.Utf8,
        "monto":        pl.Float64,
    }
)

# 2. Calcular conteo de transacciones por comercio y su giro
merchant_counts = (
    df
    .group_by(["comercio", "giro_comercio"])
    .agg([
        pl.count().alias("transacciones")
    ])
    .sort("transacciones")
)

# 3. Mostrar la tabla resultante
print("=== Transacciones por Comercio y Giro ===")

# 4. (Opcional) Exportar a CSV
merchant_counts.write_csv("../data/comercio_giro_counts.csv")
print("✅ Guardado ../data/comercio_giro_counts.csv")



=== Transacciones por Comercio y Giro ===
✅ Guardado ../data/comercio_giro_counts.csv


/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_48468/2664139158.py:5: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df = pl.read_csv(
/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_48468/2664139158.py:22: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("transacciones")
